# Problem: We want to find a N-bit number that is Maximum for our desired function

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import time

In [12]:
population_size = 20
bit_size = 5
mutation_chance = 0.1

I will define different functions in this so I can compare which one is actually better

In [103]:
class PogChampFunction():
    def __init__(self, pop_size, b_size, mut_chance):
        self.pop_size = pop_size
        self.b_size = b_size
        self.m_ch = mut_chance
        
    #This function will initialize a random population 
    def InitPop(self):
        np.random.seed(round(time.time()))  
        pop=np.random.random_integers(0, 1, (self.pop_size, self.b_size))
        return pop
    
    #I wrote two different fitness function for 2 different functions 
    def FitnessForFirstFunction(self, pop): #f(x) = 𝑥0 + 2𝑥1 + 4𝑥2 + 8𝑥3 - 16𝑥4
        fit_arr=np.zeros(self.pop_size).astype(int)
        for k in range(self.pop_size):
            fit_arr[k] = pop[k][0] + (pop[k][1] * 2) + (pop[k][2] * 4) + (pop[k][3] * 8) - (pop[k][4] * 16)
        return fit_arr
    
    def FitnessForSecondFunction(self, pop): #f(x) = 1 when u(x)≥4 f(x)= 0.6−0.2u(x) when u(x)<4,  u(x)= 𝑥0+𝑥1+𝑥2+𝑥3
        fit_arr=np.zeros(self.pop_size).astype(int)
        for k in range(self.pop_size):
            u = pop[k][0] + pop[k][1] + pop[k][2] + pop[k][3]
            if u >= 4:
                fit_arr[k] = 1
            else:
                fit_arr[k] = 0.6 - 0.2 * u
        return fit_arr
    
    #Now we get define our crossover function
    def Crossover(self, x, y):
        random.seed(a=None)
        pivot = random.randint(0, self.b_size - 1)
        t = np.zeros(self.b_size).astype(int)
        t = np.concatenate((x[0:pivot],y[pivot:self.b_size]))
        return t
    
    #this will mutate with random numbers and random value in each select random
    def MutateWithRandomNumbers(self, pop):
        random.seed(a=None)
        for k in range(self.pop_size):
            if(random.random() < self.m_ch):
                mu_count = random.randint(0, int((self.b_size-1)/2))
                t=[]
                for i in range(mu_count):
                    pos = random.randint(0, self.b_size-1)
                    if pos not in t:
                        t.append(pos)
                        pop[k][pos] = random.randint(0, 1)
        return pop
    
    #I will swap two bits
    def MutateWithSwap(self, pop):
        random.seed(a=None)
        for k in range(self.pop_size):
            if(random.random() < self.m_ch):
                f1 = 0
                f2 = 0 
                while f1 == f2:   #just to make sure we don't crossover the nodes that have been eliminated
                    f1= random.randint(0, self.b_size-1)
                    f2= random.randint(0, self.b_size-1)
                tmp = pop[k][f1]
                pop[k][f1] = pop[k][f2]                
                pop[k][f2] = tmp
        return pop
    
    #I will just reverse a single bit
    def MutateWithBitChange(self, pop):
        random.seed(a=None)
        for k in range(self.pop_size):
            if(random.random() < self.m_ch):
                pos = random.randint(0, self.b_size-1)
                if pop[k][pos] == 1:
                    pop[k][pos] = 0
                else:
                    pop[k][pos] = 1
        return pop
    
    #I will implement two random selection first I will just use the fitness score
    #Max fitness is the summation of all fitnesses
    def RandomSelectionWithFitnessPercentage(self, pop, max_fitness, fit_arr, minimum_fitness):
        x=[]
        random.seed(a=None)
        for i in range(self.pop_size):
            chance=(fit_arr[i] + 2 * abs(minimum_fitness))/(max_fitness + 2 * abs(minimum_fitness))   #since fitness can be negative we add twice the minimum to it
            if(random.random() < chance):
                x.append(i)
        return x
    
    def RandomSelectionWithRanks(self, pop, fit_arr):
        x=[]
        random.seed(a=None)
        fsr = sorted(fit_arr, reverse=True) #reverse sorted fitness 
        for i in range(self.pop_size):
            index = fsr.index(fit_arr[i])
            chance= index / ( self.pop_size)   #since fitness can be negative we add twice the minimum to it
            if(random.random() < chance):
                x.append(i)
        return x    
    

    

# For First function with bit change random selection with ranks

In [101]:
s_t=time.time()
t = PogChampFunction(population_size, bit_size, mutation_chance)
pop = t.InitPop()
k = t.FitnessForFirstFunction(pop)
old_max = np.amax(k)
ind = k.tolist().index(old_max)
old_best = pop[ind]
iterations = 0 
while iterations < 500:
    max_fitness = abs(np.sum(k))
    min_fitness = np.amin(k)
    #x = t.RandomSelectionWithFitnessPercentage(pop, max_fitness, k, min_fitness)
    x = t.RandomSelectionWithRanks(pop, k)
    f1=0
    f2=0
    for i in k:
        if len(x) > 10:
            break
        while f1 == f2 or f1 in x or f2 in x:   #just to make sure we don't crossover the nodes that have been eliminated
            f1= random.randint(0, population_size-1)
            f2= random.randint(0, population_size-1)
        if i < population_size:                  
            pop[i]=t.Crossover(pop[f1],pop[f2])
    t.MutateWithBitChange(pop)
    x.clear
    k = t.FitnessForFirstFunction(pop)
    new_max = np.amax(k)
    if new_max > old_max:
        old_max = new_max
        index = k.tolist().index(new_max)
        print(k)
        print("last best is: ", pop[index])
        iterations = 0
    else:
        iterations += 1
        
e_t=time.time()
print("run_time is:", e_t- s_t)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  # Remove the CWD from sys.path while we load stuff.


[15  2  5  5  4  8  7  7  5  3  7  2  7  4  4  8 15  6  4 -6]
last best is:  [1 1 1 1 0]
run_time is: 0.23055815696716309


# For First function with bit swap random selection with ranks

In [111]:
s_t=time.time()
t = PogChampFunction(population_size, bit_size, mutation_chance)
pop = t.InitPop()
k = t.FitnessForFirstFunction(pop)
old_max = np.amax(k)
ind = k.tolist().index(old_max)
old_best = pop[ind]
iterations = 0 
while iterations < 500:
    max_fitness = abs(np.sum(k))
    min_fitness = np.amin(k)
    #x = t.RandomSelectionWithFitnessPercentage(pop, max_fitness, k, min_fitness)
    x = t.RandomSelectionWithRanks(pop, k)
    f1=0
    f2=0
    for i in k:
        if len(x) > 10:
            break
        while f1 == f2 or f1 in x or f2 in x:   #just to make sure we don't crossover the nodes that have been eliminated
            f1= random.randint(0, population_size-1)
            f2= random.randint(0, population_size-1)
        if i < population_size:                  
            pop[i]=t.Crossover(pop[f1],pop[f2])
    t.MutateWithSwap(pop)
    x.clear
    k = t.FitnessForFirstFunction(pop)
    new_max = np.amax(k)
    if new_max > old_max:
        old_max = new_max
        index = k.tolist().index(new_max)
        print(k)
        print("last best is: ", pop[index])
        iterations = 0
    else:
        iterations += 1
        
e_t=time.time()
print("run_time is:", e_t- s_t)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  # Remove the CWD from sys.path while we load stuff.


[ 11 -15  -5   3  11 -11   3  11  -5  11  -5   3 -14  -2   3   3   3  14
   3 -14]
last best is:  [0 1 1 1 0]
[ 11 -14  -5  -5  13  11  15  14  -5  -5  -5  15 -14  11  11  15   6  14
  11   6]
last best is:  [1 1 1 1 0]
run_time is: 0.25850391387939453


# For First function with bit random and random selection with ranks

In [112]:
s_t=time.time()
t = PogChampFunction(population_size, bit_size, mutation_chance)
pop = t.InitPop()
k = t.FitnessForFirstFunction(pop)
old_max = np.amax(k)
ind = k.tolist().index(old_max)
old_best = pop[ind]
iterations = 0 
while iterations < 500:
    max_fitness = abs(np.sum(k))
    min_fitness = np.amin(k)
    #x = t.RandomSelectionWithFitnessPercentage(pop, max_fitness, k, min_fitness)
    x = t.RandomSelectionWithRanks(pop, k)
    f1=0
    f2=0
    for i in k:
        if len(x) > 10:
            break
        while f1 == f2 or f1 in x or f2 in x:   #just to make sure we don't crossover the nodes that have been eliminated
            f1= random.randint(0, population_size-1)
            f2= random.randint(0, population_size-1)
        if i < population_size:                  
            pop[i]=t.Crossover(pop[f1],pop[f2])
    t.MutateWithRandomNumbers(pop)
    x.clear
    k = t.FitnessForFirstFunction(pop)
    new_max = np.amax(k)
    if new_max > old_max:
        old_max = new_max
        index = k.tolist().index(new_max)
        print(k)
        print("last best is: ", pop[index])
        iterations = 0
    else:
        iterations += 1
        
e_t=time.time()
print("run_time is:", e_t- s_t)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  # Remove the CWD from sys.path while we load stuff.


[-16  -2   7  -8  14 -16 -16  -2 -16 -16 -16  -2  -2  -6  -2  -2  -1  -9
  -2  -2]
last best is:  [0 1 1 1 0]
[-16  -2   7  -8  -2  -6 -16  -2 -16 -15  -8  -2  -2  -2  -9  14  15  -9
  -2  -2]
last best is:  [1 1 1 1 0]
run_time is: 0.2465500831604004


# For First function with bit change random selection with Fitness function

In [115]:
s_t=time.time()
t = PogChampFunction(population_size, bit_size, mutation_chance)
pop = t.InitPop()
k = t.FitnessForFirstFunction(pop)
old_max = np.amax(k)
ind = k.tolist().index(old_max)
old_best = pop[ind]
iterations = 0 
while iterations < 500:
    max_fitness = abs(np.sum(k))
    min_fitness = np.amin(k)
    x = t.RandomSelectionWithFitnessPercentage(pop, max_fitness, k, min_fitness)
    #x = t.RandomSelectionWithRanks(pop, k)
    f1=0
    f2=0
    for i in k:
        if len(x) > 10:
            break
        while f1 == f2 or f1 in x or f2 in x:   #just to make sure we don't crossover the nodes that have been eliminated
            f1= random.randint(0, population_size-1)
            f2= random.randint(0, population_size-1)
        if i < population_size:                  
            pop[i]=t.Crossover(pop[f1],pop[f2])
    t.MutateWithBitChange(pop)
    x.clear
    k = t.FitnessForFirstFunction(pop)
    new_max = np.amax(k)
    if new_max > old_max:
        old_max = new_max
        index = k.tolist().index(new_max)
        print(k)
        print("last best is: ", pop[index])
        iterations = 0
    else:
        iterations += 1
        
e_t=time.time()
print("run_time is:", e_t- s_t)

[ -3 -11 -13  -4  -3  -3  -3   6  15  -4  14  -3  -3  -9  -1  -2  -3   5
  -3   6]
last best is:  [1 1 1 1 0]
run_time is: 0.1851644515991211


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  # Remove the CWD from sys.path while we load stuff.


# For First function with bit swap random selection with Fitness function

In [117]:
s_t=time.time()
t = PogChampFunction(population_size, bit_size, mutation_chance)
pop = t.InitPop()
k = t.FitnessForFirstFunction(pop)
old_max = np.amax(k)
ind = k.tolist().index(old_max)
old_best = pop[ind]
iterations = 0 
while iterations < 500:
    max_fitness = abs(np.sum(k))
    min_fitness = np.amin(k)
    x = t.RandomSelectionWithFitnessPercentage(pop, max_fitness, k, min_fitness)
    #x = t.RandomSelectionWithRanks(pop, k)
    f1=0
    f2=0
    for i in k:
        if len(x) > 10:
            break
        while f1 == f2 or f1 in x or f2 in x:   #just to make sure we don't crossover the nodes that have been eliminated
            f1= random.randint(0, population_size-1)
            f2= random.randint(0, population_size-1)
        if i < population_size:                  
            pop[i]=t.Crossover(pop[f1],pop[f2])
    t.MutateWithSwap(pop)
    x.clear
    k = t.FitnessForFirstFunction(pop)
    new_max = np.amax(k)
    if new_max > old_max:
        old_max = new_max
        index = k.tolist().index(new_max)
        print(k)
        print("last best is: ", pop[index])
        iterations = 0
    else:
        iterations += 1
        
e_t=time.time()
print("run_time is:", e_t- s_t)

[  1   5   5  12   6   5   5   5   5   5   5   8   5 -16 -14   9   8  13
 -10   4]
last best is:  [1 0 1 1 0]
[ 12  12  14  12  14   0  -8 -12  12  14  12   2  12  14   6   9   8  13
 -10   4]
last best is:  [0 1 1 1 0]
[-12  13  15  -2  -2  -2   5  -5  -9  -2  -2   4   4  14  -2   3  -3  11
  -2   8]
last best is:  [1 1 1 1 0]
run_time is: 0.1851494312286377


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  # Remove the CWD from sys.path while we load stuff.


# For First function with random bit, random selection with Fitness function

In [119]:
s_t=time.time()
t = PogChampFunction(population_size, bit_size, mutation_chance)
pop = t.InitPop()
k = t.FitnessForFirstFunction(pop)
old_max = np.amax(k)
ind = k.tolist().index(old_max)
old_best = pop[ind]
iterations = 0 
while iterations < 500:
    max_fitness = abs(np.sum(k))
    min_fitness = np.amin(k)
    x = t.RandomSelectionWithFitnessPercentage(pop, max_fitness, k, min_fitness)
    #x = t.RandomSelectionWithRanks(pop, k)
    f1=0
    f2=0
    for i in k:
        if len(x) > 10:
            break
        while f1 == f2 or f1 in x or f2 in x:   #just to make sure we don't crossover the nodes that have been eliminated
            f1= random.randint(0, population_size-1)
            f2= random.randint(0, population_size-1)
        if i < population_size:                  
            pop[i]=t.Crossover(pop[f1],pop[f2])
    t.MutateWithRandomNumbers(pop)
    x.clear
    k = t.FitnessForFirstFunction(pop)
    new_max = np.amax(k)
    if new_max > old_max:
        old_max = new_max
        index = k.tolist().index(new_max)
        print(k)
        print("last best is: ", pop[index])
        iterations = 0
    else:
        iterations += 1
        
e_t=time.time()
print("run_time is:", e_t- s_t)

[-12  13  14   5 -15  12  12  -2   8   8  12  12   8   8  -1  12   8   2
  12  12]
last best is:  [0 1 1 1 0]
[-12  13  14   5 -15  13  12  -2  15   8  12  12  13  15  15  12   8   2
  -4  13]
last best is:  [1 1 1 1 0]
run_time is: 0.18965792655944824


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  # Remove the CWD from sys.path while we load stuff.


# Now we go for 2nd function 

In [124]:
bit_size=4

# For second function with bit change random selection with ranks

In [129]:
s_t=time.time()
t = PogChampFunction(population_size, bit_size, mutation_chance)
pop = t.InitPop()
k = t.FitnessForSecondFunction(pop)
old_max = np.amax(k)
ind = k.tolist().index(old_max)
old_best = pop[ind]
iterations = 0 
while iterations < 500:
    max_fitness = abs(np.sum(k))
    min_fitness = np.amin(k)
    #x = t.RandomSelectionWithFitnessPercentage(pop, max_fitness, k, min_fitness)
    x = t.RandomSelectionWithRanks(pop, k)
    f1=0
    f2=0
    for i in k:
        if len(x) > 10:
            break
        while f1 == f2 or f1 in x or f2 in x:   #just to make sure we don't crossover the nodes that have been eliminated
            f1= random.randint(0, population_size-1)
            f2= random.randint(0, population_size-1)
        if i < population_size:                  
            pop[i]=t.Crossover(pop[f1],pop[f2])
    t.MutateWithBitChange(pop)
    x.clear
    k = t.FitnessForSecondFunction(pop)
    new_max = np.amax(k)
    if new_max > old_max:
        old_max = new_max
        index = k.tolist().index(new_max)
        print(k)
        print("last best is: ", pop[index])
        iterations = 0
    else:
        iterations += 1
        
e_t=time.time()
print("run_time is:", e_t- s_t)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  # Remove the CWD from sys.path while we load stuff.


[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
last best is:  [1 1 1 1]
run_time is: 0.26999568939208984


# For second function with bit swap random selection with ranks

In [133]:
s_t=time.time()
t = PogChampFunction(population_size, bit_size, mutation_chance)
pop = t.InitPop()
k = t.FitnessForSecondFunction(pop)
old_max = np.amax(k)
ind = k.tolist().index(old_max)
old_best = pop[ind]
iterations = 0 
while iterations < 500:
    max_fitness = abs(np.sum(k))
    min_fitness = np.amin(k)
    #x = t.RandomSelectionWithFitnessPercentage(pop, max_fitness, k, min_fitness)
    x = t.RandomSelectionWithRanks(pop, k)
    f1=0
    f2=0
    for i in k:
        if len(x) > 10:
            break
        while f1 == f2 or f1 in x or f2 in x:   #just to make sure we don't crossover the nodes that have been eliminated
            f1= random.randint(0, population_size-1)
            f2= random.randint(0, population_size-1)
        if i < population_size:                  
            pop[i]=t.Crossover(pop[f1],pop[f2])
    t.MutateWithSwap(pop)
    x.clear
    k = t.FitnessForSecondFunction(pop)
    new_max = np.amax(k)
    if new_max > old_max:
        old_max = new_max
        index = k.tolist().index(new_max)
        print(k)
        print("last best is: ", pop[index])
        iterations = 0
    else:
        iterations += 1
        
e_t=time.time()
print("run_time is:", e_t- s_t)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  # Remove the CWD from sys.path while we load stuff.


[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
last best is:  [1 1 1 1]
run_time is: 0.2624974250793457


# For Second function with bit random and random selection with ranks

In [134]:
s_t=time.time()
t = PogChampFunction(population_size, bit_size, mutation_chance)
pop = t.InitPop()
k = t.FitnessForSecondFunction(pop)
old_max = np.amax(k)
ind = k.tolist().index(old_max)
old_best = pop[ind]
iterations = 0 
while iterations < 500:
    max_fitness = abs(np.sum(k))
    min_fitness = np.amin(k)
    #x = t.RandomSelectionWithFitnessPercentage(pop, max_fitness, k, min_fitness)
    x = t.RandomSelectionWithRanks(pop, k)
    f1=0
    f2=0
    for i in k:
        if len(x) > 10:
            break
        while f1 == f2 or f1 in x or f2 in x:   #just to make sure we don't crossover the nodes that have been eliminated
            f1= random.randint(0, population_size-1)
            f2= random.randint(0, population_size-1)
        if i < population_size:                  
            pop[i]=t.Crossover(pop[f1],pop[f2])
    t.MutateWithRandomNumbers(pop)
    x.clear
    k = t.FitnessForSecondFunction(pop)
    new_max = np.amax(k)
    if new_max > old_max:
        old_max = new_max
        index = k.tolist().index(new_max)
        print(k)
        print("last best is: ", pop[index])
        iterations = 0
    else:
        iterations += 1
        
e_t=time.time()
print("run_time is:", e_t- s_t)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  # Remove the CWD from sys.path while we load stuff.


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
last best is:  [1 1 1 1]
run_time is: 0.26999855041503906


# For second function with bit change random selection with Fitness function

In [138]:
s_t=time.time()
t = PogChampFunction(population_size, bit_size, mutation_chance)
pop = t.InitPop()
k = t.FitnessForSecondFunction(pop)
old_max = np.amax(k)
ind = k.tolist().index(old_max)
old_best = pop[ind]
iterations = 0 
while iterations < 500:
    max_fitness = abs(np.sum(k))
    min_fitness = np.amin(k)
    x = t.RandomSelectionWithFitnessPercentage(pop, max_fitness, k, min_fitness)
    #x = t.RandomSelectionWithRanks(pop, k)
    f1=0
    f2=0
    for i in k:
        if len(x) > 10:
            break
        while f1 == f2 or f1 in x or f2 in x:   #just to make sure we don't crossover the nodes that have been eliminated
            f1= random.randint(0, population_size-1)
            f2= random.randint(0, population_size-1)
        if i < population_size:                  
            pop[i]=t.Crossover(pop[f1],pop[f2])
    t.MutateWithBitChange(pop)
    x.clear
    k = t.FitnessForSecondFunction(pop)
    new_max = np.amax(k)
    if new_max > old_max:
        old_max = new_max
        index = k.tolist().index(new_max)
        print(k)
        print("last best is: ", pop[index])
        iterations = 0
    else:
        iterations += 1
        
e_t=time.time()
print("run_time is:", e_t- s_t)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in long_scalars


[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
last best is:  [1 1 1 1]
run_time is: 0.2605001926422119


# For second function with bit swap random selection with Fitness function

In [142]:
s_t=time.time()
t = PogChampFunction(population_size, bit_size, mutation_chance)
pop = t.InitPop()
k = t.FitnessForSecondFunction(pop)
old_max = np.amax(k)
ind = k.tolist().index(old_max)
old_best = pop[ind]
iterations = 0 
while iterations < 500:
    max_fitness = abs(np.sum(k))
    min_fitness = np.amin(k)
    x = t.RandomSelectionWithFitnessPercentage(pop, max_fitness, k, min_fitness)
    #x = t.RandomSelectionWithRanks(pop, k)
    f1=0
    f2=0
    for i in k:
        if len(x) > 10:
            break
        while f1 == f2 or f1 in x or f2 in x:   #just to make sure we don't crossover the nodes that have been eliminated
            f1= random.randint(0, population_size-1)
            f2= random.randint(0, population_size-1)
        if i < population_size:                  
            pop[i]=t.Crossover(pop[f1],pop[f2])
    t.MutateWithSwap(pop)
    x.clear
    k = t.FitnessForSecondFunction(pop)
    new_max = np.amax(k)
    if new_max > old_max:
        old_max = new_max
        index = k.tolist().index(new_max)
        print(k)
        print("last best is: ", pop[index])
        iterations = 0
    else:
        iterations += 1
        
e_t=time.time()
print("run_time is:", e_t- s_t)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in long_scalars


[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
last best is:  [1 1 1 1]
run_time is: 0.3049447536468506


# For Second function with random bit, random selection with Fitness function

In [145]:
s_t=time.time()
t = PogChampFunction(population_size, bit_size, mutation_chance)
pop = t.InitPop()
k = t.FitnessForSecondFunction(pop)
old_max = np.amax(k)
ind = k.tolist().index(old_max)
old_best = pop[ind]
iterations = 0 
while iterations < 500:
    max_fitness = abs(np.sum(k))
    min_fitness = np.amin(k)
    x = t.RandomSelectionWithFitnessPercentage(pop, max_fitness, k, min_fitness)
    #x = t.RandomSelectionWithRanks(pop, k)
    f1=0
    f2=0
    for i in k:
        if len(x) > 10:
            break
        while f1 == f2 or f1 in x or f2 in x:   #just to make sure we don't crossover the nodes that have been eliminated
            f1= random.randint(0, population_size-1)
            f2= random.randint(0, population_size-1)
        if i < population_size:                  
            pop[i]=t.Crossover(pop[f1],pop[f2])
    t.MutateWithRandomNumbers(pop)
    x.clear
    k = t.FitnessForSecondFunction(pop)
    new_max = np.amax(k)
    if new_max > old_max:
        old_max = new_max
        index = k.tolist().index(new_max)
        print(k)
        print("last best is: ", pop[index])
        iterations = 0
    else:
        iterations += 1
        
e_t=time.time()
print("run_time is:", e_t- s_t)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in long_scalars


[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
last best is:  [1 1 1 1]
run_time is: 0.28748631477355957
